In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.2.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.2.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/spark-3.2.0-bin-hadoop3.2"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-01-02 00:26:30--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.46MB/s    in 0.2s    

2022-01-02 00:26:30 (4.46 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Shoes").getOrCreate()

In [4]:
from pyspark import SparkFiles
# Load in user_data.csv from S3 into a DataFrame
url3 = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Shoes_v1_00.tsv.gz"
spark.sparkContext.addFile(url3)

df3 = spark.read.option("delimiter", "\t").option("header","True").option("InferSchema","True").csv(SparkFiles.get("amazon_reviews_us_Shoes_v1_00.tsv.gz"))
df3.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18069663|R3P2HIOQCIN5ZU|B000XB31C0|     265024781|Minnetonka Men's ...|           Shoes|          1|            0|          0|   N|                Y|                   .|Do not buy: reall...| 2015-08-31|
|         US|   16251825|R12VVR0WH5Q24V|B00CFYZH5W|     259035853|Teva Men's Pajaro...|           Shoes|          5|    

In [5]:
#count the number of rows in the dataset
df3.count()

4366916

In [6]:
#Transform the dataset to fit the tables in the schema file. Be sure the DataFrames match in data type and in column name.
#create table review_id_table
review_id_table = df3.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_table.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R3P2HIOQCIN5ZU|   18069663|B000XB31C0|     265024781| 2015-08-31|
|R12VVR0WH5Q24V|   16251825|B00CFYZH5W|     259035853| 2015-08-31|
| RNCCKB6TV5EEF|   20381037|B00S8JNN3Q|     666066660| 2015-08-31|
|R2NZXYIVCGB13W|     108364|B00XFBPOQG|     448483263| 2015-08-31|
|R2EQ1TG9IT3OEQ|   45449350|B00SW64Y9W|       7853171| 2015-08-31|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [7]:
#Create table products
products = df3.select(["product_id","product_title"])
products.show(5)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B000XB31C0|Minnetonka Men's ...|
|B00CFYZH5W|Teva Men's Pajaro...|
|B00S8JNN3Q|Anne Klein Perfec...|
|B00XFBPOQG|adidas Men's 10K ...|
|B00SW64Y9W|OverBling Sneaker...|
+----------+--------------------+
only showing top 5 rows



In [8]:
#create table customers
customers = df3.groupby("customer_id").count()
customers = customers.withColumnRenamed("count","customer_count")
customers.show(5)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   16121565|             5|
|   48146680|             1|
|   11062912|             1|
|   51451778|             1|
|   27920838|             1|
+-----------+--------------+
only showing top 5 rows



In [9]:
#create table vine_table
vine_table = df3.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_table.show(5)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R3P2HIOQCIN5ZU|          1|            0|          0|   N|
|R12VVR0WH5Q24V|          5|            0|          0|   N|
| RNCCKB6TV5EEF|          4|            0|          0|   N|
|R2NZXYIVCGB13W|          5|            0|          6|   N|
|R2EQ1TG9IT3OEQ|          3|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



In [ ]:
 # Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://<instance>.rds.amazonaws.com:5432/bigdata"
config = {"user":"root", 
          "password": "<password>", 
          "driver":"org.postgresql.Driver"}

In [ ]:
#write tables in rds
review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)
products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)
customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)
vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)